In [15]:
import torch
import torchvision
import torchvision.transforms as transform
import torch.nn as nn
import torch.nn.functional as F

In [11]:
transform = transform.Compose(
    [transform.ToTensor(),
     transform.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))]
)
trainset = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transform)

Files already downloaded and verified


In [12]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [13]:
# num_worker -> 加載batch的現成數量，越多越快。
# shuffle -> 每個epoch都會重新調整input data的次序
# batch_size -> 是指一次訓練要多少 data進去訓練，之後訓練data_size / batch_size次，就算一個epoch（也就是全部run過一次）。
# batch_size 越大，epoch需要越多，因為訓練次數會變少。 epoch像是組數，batch_size像是重量，不能太大，也不能太小。
trainloader = torch.utils.data.DataLoader( trainset, batch_size = 4, shuffle = True, num_workers = 2)
testset = torchvision.datasets.CIFAR10( root = './data', train = False, download = True, transform = transform)

Files already downloaded and verified


In [7]:
trainset

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [14]:
testloader = torch.utils.data.DataLoader( testset, batch_size = 4, shuffle = False, num_workers = 2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [52]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # super的用處就是調用father class的function。在這裡就是使用Father class的__init__()
        self.conv1 = nn.Conv2d(3, 6, 5) # input:3, output:6, kernel:5
        self.pool = nn.MaxPool2d(2, 2) # kernel:2, stride:2
        self.conv2 = nn.Conv2d(6, 16, 5) # input:6, output:16, kernel:5
        self.fc1 = nn.Linear(16 * 5 * 5, 120) # in->400, output->120
        self.fc2 = nn.Linear(120,84) # in->120, output->84
        self.fc3 = nn.Linear(84,10) # in->84, output->10
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()

In [30]:
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [33]:
print(net.parameters())

<generator object Module.parameters at 0x0000023681AC2AB0>


In [44]:
import torch.optim as optim

# 什麼是Cross Entropy Loss?
criterion = nn.CrossEntropyLoss()
# 什麼是optimizer, 用處是什麼？
optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum = 0.9)

In [47]:
# print(vars(optimizer))

In [53]:
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 獲得輸入的資料跟答案
        inputs, labels = data
        # print(f"input = {inputs}")
        # print(f"label = {labels}")
        # 把梯度先設成 0
        optimizer.zero_grad()
        
        # 正向傳播：傳入訓練資料
        outputs = net(inputs)
        loss = criterion(outputs, labels) # 計算loss值
        # 反向傳播
        loss.backward()
        # 優化
        optimizer.step()
        
        # 秀出現在的狀態資訊
        running_loss += loss.item()
        if i%2000 == 1999:
            print("[%d, %5d] loss: %.3f" %(epoch + 1, i+1, running_loss/2000))
            running_loss = 0.0
                  
print('Finish Training')

[1,  2000] loss: 2.303
[1,  4000] loss: 2.304
[1,  6000] loss: 2.304
[1,  8000] loss: 2.303
[1, 10000] loss: 2.304
[1, 12000] loss: 2.304
[2,  2000] loss: 2.303
[2,  4000] loss: 2.304
[2,  6000] loss: 2.303
[2,  8000] loss: 2.303
[2, 10000] loss: 2.303
[2, 12000] loss: 2.304
Finish Training


In [54]:
dataiter = iter(testloader)
images, labels = dataiter.next()

AttributeError: '_MultiProcessingDataLoaderIter' object has no attribute 'next'

In [ ]:
imshow